In [ ]:
import time

import numpy as np

import tools.rotations as rot

from controller import locomotion
import mujoco

import copy
%load_ext autoreload
%autoreload 2

# real robot structure, 5 fingers, 4Dofs for each finger

In [ ]:
xml_path = 'descriptions/v2/hand_v2.xml'
q = np.array([0,0,  0.13,  9.99999738e-01,
       -4.46340938e-04,  5.67392530e-04, -4.89152197e-05, 
                0.,    7.91144283e-01,  0.3,  0.3,
               0.3,   7.91144283e-01,  0.3,  0.3,
               0.,    7.91144283e-01,  0.3,  0.3,
               0.,    7.91144283e-01,  0.3,  0.3,
              -0.5,   7.91144283e-01,  0.3,  0.3
            ])
obj_names = ['sphere_1','cylinder_1', 'box_1', 'cube_1', ]
# obj_names = ['box_1', 'cube_1']
r = locomotion(xml_path, q0=q, finger_num=5, path_suffix='../',obj_names=obj_names, v='v2')

print('locomotion freq', r.f)
# take a rest to let the acceleration converge to zero
t0 = time.time()
q0 = copy.deepcopy(q[7:])
while 1:
    r.joint_computed_torque_control(q0)
    if time.time() - t0 > 4:
        break
r.xh_local_init = copy.deepcopy(r.xh_local) 

In [ ]:
r.obj_names

In [ ]:
obj_1st = 0
offset = np.array([0.2, 0])  # offset along xy axis
 
attractor_1 = r.x_obj[obj_1st][:2] + offset
print(attractor_1)


In [ ]:


i = 2  # the object to be grasped
radius = [0.2, 0.2, 0.3, 0.3]

attractor_1 = r.x_obj[i][:3] + rot.quat2mat(r.x_obj[i][3:7]) @ np.array([0, 0 ,0.35])   # offset along z axis of the obj frame, y is the long size of the box
# attractor_1 = attractor_1[:2]
# attractor_1 = np.array([-1,0])
ori_d = rot.euler2quat([0,0, np.pi/2 + np.pi/2/4])
ori_tmp = rot.quat_mul(r.x_obj[i][3:7], rot.euler2quat([-np.pi/2, 0, np.pi/2]))
ori_d = rot.quat_mul(ori_d, ori_tmp)


ref = np.concatenate([attractor_1, ori_d])
# visualize the attractor
r.d.mocap_pos[:] = ref[:3]
r.d.mocap_quat[:] = ref[3:]
# r.step()
r.moveto_attractor(ref, threshold=0.015, stance_legs=[0,2,3], swing_legs=[1,4], max_vel=0.005, radius=radius)



In [ ]:
# prepare to grasp

i = 2  # the object index to be grasped
radius = [0.2, 0.2, 0.03, 0.3]

attractor_1 = r.x_obj[i][:3] + rot.quat2mat(r.x_obj[i][3:7]) @ np.array([0, 0 ,0.15])   # offset along z axis of the obj frame, y is the long size of the box

ori_d = rot.euler2quat([0,0, np.pi/2 + np.pi/2/4])
ori_tmp = rot.quat_mul(r.x_obj[i][3:7], rot.euler2quat([-np.pi/2, 0, np.pi/2]))
ori_d = rot.quat_mul(ori_d, ori_tmp)


ref = np.concatenate([attractor_1, ori_d])
# visualize the attractor
r.d.mocap_pos[:] = ref[:3]
r.d.mocap_quat[:] = ref[3:]
# r.step()
r.moveto_attractor(ref, threshold=0.015, stance_legs=[0,2,3], swing_legs=[1,4], max_vel=0.005, radius=radius)


In [ ]:
#  change the standing configuration
# _ = r.move_along_z(z=-0.04)
q_cmd = copy.deepcopy(q0)
q_cmd[4] = -0.5
q_cmd[16] = 0.5
r.joint_motion_generator(q_cmd, vel=2)

In [ ]:
# 3 grasping
q_23 = np.array([-0.4,   7.91144283e-01,  0.3,  0.3,
                 0.4,   7.91144283e-01,  0.3,  0.3])
q_cmd = copy.deepcopy(q0)
q_cmd[8:16] = q_23
q_cmd[4] = -0.5
q_cmd[16] = 0.5
nums = 2000
q_23_list = np.linspace(q0[8:16], q_23, num=nums)


r.joint_motion_generator(q_cmd, vel=1)


In [ ]:
q_23 = np.array([-0.5,   -1.4,  -1.3,  -0.3,
                 0.5,   -1.4,  -1.3,  -0.3])
q_cmd = copy.deepcopy(q0)
q_cmd[8:16] = q_23
q_cmd[4] = -0.5
q_cmd[16] = 0.5

r.joint_motion_generator(q_cmd, vel=1)

    